## IMPORTS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
from collections import Counter
import ast
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

from datetime import datetime
from openpyxl import load_workbook
from openpyxl.utils.exceptions import InvalidFileException

from transformers import (
    BertTokenizer, 
    RobertaForSequenceClassification, 
    RobertaTokenizer, 
    RobertaModel, 
    AutoModel, 
    AutoTokenizer, 
    XLMRobertaForSequenceClassification, 
    XLMRobertaTokenizer,
    Trainer, 
    TrainingArguments, 
    BertForSequenceClassification, 
    TrainerCallback
)

import datasets
from datasets import Dataset
import pyarrow as pa
import pyarrow.dataset as ds

import random

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from sklearn.metrics import precision_score, recall_score, f1_score


In [ ]:
print('Downlowing stopwords...')
nltk.download('stopwords')
nltk.download('punkt')

## FUNCIONES

Funciones que utilizamos para el propocesamiento de los datos: remove_stopwords, stemming, stemming_turkish y clean

In [ ]:
def remove_stopwords(text, language):

    # Cargamos las stopwords
    if language == 'english':
            stop_words = set(stopwords.words('english'))
    elif language == 'spanish':
            stop_words = set(stopwords.words('spanish'))
    elif language == 'turkish':
            stop_words = set(nltk.corpus.stopwords.words('turkish'))
    else:
      print('Language not supported')

    # Eliminamos las stopwords
    cleanTxt = [x for x in text.split() if x not in stop_words]
    return ' '.join(cleanTxt)

def stemming(text, language):
    stemmer = SnowballStemmer(language)
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

# Definimos las función stemming para el turco
def stemming_turkish(text):
    stemmer_tr = snowballstemmer.stemmer('turkish')
    words = text.split()
    stemmed_words = [stemmer_tr.stemWord(word) for word in words]
    return ' '.join(stemmed_words)

# Define the cleaning function
def clean(text, language):

    if isinstance(text, str):
        cleanTxt = re.sub(r'https?://\S+|www\.\S+', '', text)  # Eliminamos las URLs
        cleanTxt = re.sub(r'#\w+', '', cleanTxt)               # Eliminamos los hashtags
        cleanTxt = re.sub(r'\W+|\d+', ' ', cleanTxt)           # Eliminamos los carácteres y números no alphanumericos and numbers
        cleanTxt = cleanTxt.lower()                            # Minúsculas
    else:
        # Convertimos el elemento a cadena de texto
        text = str(text)
        cleanTxt = re.sub(r'https?://\S+|www\.\S+', '', text)  # Eliminamos las URLs
        cleanTxt = re.sub(r'#\w+', '', cleanTxt)               # Eliminamos los hashtags
        cleanTxt = re.sub(r'\W+|\d+', ' ', cleanTxt)           # Eliminamos los carácteres y números no alphanumericos and numbers
        cleanTxt = cleanTxt.lower()                            # Minúsculas
        
    # Stopwords 
    stop_words = set(stopwords.words(language))
    cleanTxt = ' '.join([word for word in cleanTxt.split() if word not in stop_words])

    # Stemming
    if language == 'turkish':
        cleanTxt = stemming_turkish(cleanTxt)
    else:
        cleanTxt = stemming(cleanTxt, language)

    return cleanTxt


In [ ]:
def load_data(path, task, language):
    data = pd.read_csv(path, sep='\t')

    if language == 'english':
        data = data[data['language'] == 'en']

    elif language == 'spanish':
        data = data[data['language'] == 'es']

    else:
      print('not supported')

    data['text']  = data['text'].apply(clean, language=language)
    data = pd.DataFrame({'text': data['text'], 'label': data[task]})     
    if task == 'task1':
        data['label']  = data['label'].map({'sexist': 1, 'non-sexist': 0})  # Convertimos en una variable binaria

    if task == 'task2':
        data['label']  = data['label'].map({'ideological-inequality': 1,    # Convertimos en 6 clases numéricas
                                            'stereotyping-dominance': 2,
                                            'objectification': 3,
                                            'sexual-violence': 4,
                                            'misogyny-non-sexual-violence': 5,
                                            'non-sexist': 0})

    return data


# Cargamos el dataset turco
def load_turkish_data(path, task, language):
    df_exist_turkish = pd.read_csv(path, delimiter=';')
    df_exist_turkish = df_exist_turkish.rename(columns={
        'Annotation_L1_Eng': 'task1',
        'Annotation_L2_Eng': 'task2',
        'Annotation_L1_TR': 'task1_tr',
        'Annotation_L2_TR': 'task2_tr',
        'Text': 'text'
    })

    # Convertimos todos los elementos a minúsculas
    df_exist_turkish = df_exist_turkish.apply(lambda col: col.map(lambda x: x.lower() if isinstance(x, str) else x))

    # Añadimos la columna del idioma
    df_exist_turkish['language'] = 'tr'

    # Renombramos para unificar los datasets
    replacements = {
        'not-sexist': 'non-sexist',
        'anti-feminism': 'ideological-inequality',
        'misogyny': 'misogyny-non-sexual-violence',
        'sexual_violence': 'sexual-violence',
        'stereotyping': 'stereotyping-dominance'
    }
    df_exist_turkish.replace(replacements, inplace=True)

    # Eliminamos las columnas innecesarias
    df_exist_turkish.drop(['task1_tr', 'task2_tr', 'ID'], axis=1, inplace=True)

    df_exist_turkish['text'] = df_exist_turkish['text'].apply(lambda x: clean(x, language))

    data = pd.DataFrame({'text': df_exist_turkish['text'], 'label': df_exist_turkish[task]})        
    if task == 'task1':
        data['label']  = data['label'].map({'sexist': 1, 'non-sexist': 0})  # Convertimos en una variable binaria

    if task == 'task2':
        data['label']  = data['label'].map({'ideological-inequality': 1,    # Convertimos en 6 clases numéricas
                                            'stereotyping-dominance': 2,
                                            'objectification': 3,
                                            'sexual-violence': 4,
                                            'misogyny-non-sexual-violence': 5,
                                            'non-sexist': 0})


    return data


In [ ]:
# Cargamos el tokenizador
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

# Función que tokeniza el modelo
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = True, truncation=True)

def formatting_data(df):
    if not all(isinstance(text, str) for text in df['text']):
        df['text'] = df['text'].astype(str)  # Convertimos los valores no str a str
    
    data = Dataset(pa.Table.from_pandas(df))                                    # Convertimos el DataFrame en un conjunto de datos de Huggingface
    data = data.map(lambda batch: tokenization(batch, language), batched=True, batch_size=len(data))  # Aplicamos la funcion tokenizador
    data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])  # Establecemos el formato
    return data



Creamos una función para guardar los datos en excels

In [ ]:
class SaveMetricsCallback(TrainerCallback):
    def __init__(self, file_name):
        self.metrics_data = []
        self.file_name = file_name

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        # Guardamos las métricas después de cada epoch
        if metrics is not None:
            self.metrics_data.append(metrics)

    def save_to_excel(self):
        # Creamos un DataFrame con las métricas
        df = pd.DataFrame(self.metrics_data)

        # Definimos la ruta
        file_path = f'{self.file_name}.xlsx'
        valor = random.random()
        sheet_name = f"hoja_{valor}"

        try:
            # Cargamos el libro existente
            try:
                book = load_workbook(file_path)
                # Verificamos si la hoja existe
                if sheet_name in book.sheetnames:
                    # Si la hoja existe la eliminamos
                    del book[sheet_name]
                    book.save(file_path)
                # Guardamos los datos
                with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
            except (FileNotFoundError, InvalidFileException, KeyError):
                # Si el libro no se puede cargar o está vacío, creamos uno nuevo
                with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
                    df.to_excel(writer, sheet_name=sheet_name, index=False)

            print(f"Datos guardados en el archivo '{self.file_name}.xlsx'.")

        except PermissionError:
            print(f"Error: No se pudo acceder o guardar en '{self.file_name}.xlsx'.")



Función auxiliar que imprime las métricas que analizaremos en nuestro modelo y nos ayudará a sacar conclusiones

In [ ]:
def metrics(predicted):
    labels = predicted.label_ids
    predictions = predicted.predictions.argmax(-1)

    #Calcula la precisión del clasificador
    accuracy = accuracy_score(labels, predictions)
    print('Accuracy:', accuracy)

    report = classification_report(labels, predictions, output_dict=True, zero_division=0)
    precision_per_class = {label: metrics['precision'] for label, metrics in report.items() if isinstance(metrics, dict)}
    recall_per_class = {label: metrics['recall'] for label, metrics in report.items() if isinstance(metrics, dict)}
    f1_score_per_class = {label: metrics['f1-score'] for label, metrics in report.items() if isinstance(metrics, dict)}

    precision = report['weighted avg']['precision']
    recall = recall_score(labels, predictions, average='macro', zero_division=0)
    f1 = report['weighted avg']['f1-score']
   
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}


## XLM ROBERTA 

### task 1

## Cargamos los datos

In [ ]:
# english
train_load_en = load_data("EXIST_training.tsv", 'task1', 'english')
test_load_en = load_data("EXIST_test_labeled.tsv", 'task1', 'english')
#spanish
train_load_es = load_data("EXIST_training.tsv", 'task1', 'spanish')
test_load_es = load_data("EXIST_test_labeled.tsv",  'task1', 'spanish')
#turkish
df_exist_turkish = load_turkish_data("Dataset_Sexism_Turkish.csv",  'task1', 'turkish')

In [ ]:
#separamos el dataset en training y test set
train_load_tr, test_load_tr = train_test_split(df_exist_turkish, test_size=0.4, random_state=42)

In [ ]:
train_load_en , validation_load_en  = train_test_split(train_load_en , test_size=0.2, random_state=42) #english
train_load_es , validation_load_es  = train_test_split(train_load_es , test_size=0.2, random_state=42) #spanish
train_load_tr , validation_load_tr  = train_test_split(train_load_tr , test_size=0.2, random_state=42) #turkish

Juntamos los 3 datasets, para obtener uno único y entrenar así nuestro modelo con el conjunto de datos multilingue

In [ ]:
train_load = pd.concat([train_load_en, train_load_es, train_load_tr], ignore_index=True)
test_load = pd.concat([test_load_en, test_load_es, test_load_tr], ignore_index=True)
validation_load = pd.concat([validation_load_en, validation_load_es, validation_load_tr], ignore_index=True)

In [ ]:
# Todos los idiomas
train = formatting_data(train_load)
test  = formatting_data(test_load )
validation = formatting_data(validation_load )

In [ ]:
# Cargamos el modelo
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base")

In [ ]:
# Definimos los arumentos de entrenamientos de los modelos 
training_args = TrainingArguments(
    output_dir='RESULTADOS',
    evaluation_strategy='epoch',
    logging_strategy='steps',  # Para una evaluación más frecuente
    save_strategy='epoch',
    learning_rate=3e-5,  # Tasa de aprendizaje
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,  
    num_train_epochs=10, 
    weight_decay=0.1, 
    logging_dir='LOGS',
    logging_steps=100,  
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    save_total_limit=3,  # Limitamos el número de modelos guardados
    gradient_accumulation_steps=2,  
)

In [ ]:
# Metricas
metrics_callback = SaveMetricsCallback(file_name='training_metrics_XLM_t1')

# Trainer class
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=metrics,
    train_dataset=train,
    eval_dataset=validation,
    callbacks=[metrics_callback]
)

# Entrenamos el modelo
print('\n* TRAIN *\n')
trainer.train()

# Evaluamos el model
print('\n* EVALUATE *\n')
trainer.evaluate(test)

# Guardamos las métricas
metrics_callback.save_to_excel()

In [ ]:
# Guardamos el modelo y el tokenizador
model_name = 'saved_xlm_roberta_model_task1'
model.save_pretrained(model_name)
tokenizer.save_pretrained(model_name)

### task 2

## Cargamos los datos

In [ ]:
# english
train_load_en = load_data("EXIST2021_training.tsv", 'task2', 'english')
test_load_en = load_data("EXIST2021_test_labeled.tsv", 'task2', 'english')
#spanish
train_load_es = load_data("EXIST2021_training.tsv", 'task2', 'spanish')
test_load_es = load_data("EXIST2021_test_labeled.tsv",  'task2', 'spanish')
#turkish
df_exist_turkish = load_turkish_data("Dataset_Sexism_Turkish.csv",  'task2', 'turkish')

In [ ]:
#separamos el dataset en training y test set
train_load_tr, test_load_tr = train_test_split(df_exist_turkish, test_size=0.4, random_state=42)

In [ ]:
train_load_en , validation_load_en  = train_test_split(train_load_en , test_size=0.2, random_state=42) #english
train_load_es , validation_load_es  = train_test_split(train_load_es , test_size=0.2, random_state=42) #spanish
train_load_tr , validation_load_tr  = train_test_split(train_load_tr , test_size=0.2, random_state=42) #turkish

In [ ]:
train_load = pd.concat([train_load_en, train_load_es, train_load_tr], ignore_index=True)
test_load = pd.concat([test_load_en, test_load_es, test_load_tr], ignore_index=True)
validation_load = pd.concat([validation_load_en, validation_load_es, validation_load_tr], ignore_index=True)

In [ ]:
# Todos los idiomas
train = formatting_data(train_load)
test  = formatting_data(test_load )
validation = formatting_data(validation_load )

In [ ]:
# Cargamos el modelo
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base")

In [ ]:
training_args_2 = TrainingArguments(
    output_dir='RESULTADOS',
    evaluation_strategy='epoch',
    logging_strategy='steps',  
    save_strategy='epoch',
    learning_rate=3e-5, 
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32, 
    num_train_epochs=10,  
    weight_decay=0.1, 
    logging_dir='LOGS',
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    save_total_limit=3, 
    gradient_accumulation_steps=2,  
)

In [ ]:
metrics_callback = SaveMetricsCallback(file_name='training_metrics_XLM_t2')

trainer = Trainer(
    model=model,
    args=training_args_2,
    compute_metrics=metrics,
    train_dataset=train,
    eval_dataset=validation,
    callbacks=[metrics_callback]
)

# Entrenamos el modelo
print('\n* TRAIN *\n')
trainer.train()

# Evaluamos el model
print('\n* EVALUATE *\n')
trainer.evaluate(test)

# Guardamos las métricas
metrics_callback.save_to_excel()

In [ ]:
# Save the model and tokenizer
model_name = 'saved_xlm_roberta_model_task2'
model.save_pretrained(model_name)
tokenizer.save_pretrained(model_name)
